In [1]:
from pathlib import Path
import sys
sys.path.insert(0, str(Path("../src").resolve()))

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


import optuna
optuna.logging.disable_default_handler()
from optuna.visualization import plot_edf
from optuna.visualization import plot_slice
from optuna.visualization import plot_contour
from optuna.visualization import plot_parallel_coordinate


from itertools import product
import pandas as pd
import numpy as np
import os


import src.utils.config as config
from src.modelling.optimize import (
generate_opt_data,
days_to_event,
optimize,
wrapper,
plot_animation
)


import warnings
warnings.filterwarnings('ignore')

/Users/anand_k/Downloads/Laxmi/Symphony/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [12]:
config_catalog = config.load_catalog_params(os.getcwd(), 'conf/catalogs')
config_parameters = config.load_catalog_params(os.getcwd(), 'conf/parameters')
version = str(config_catalog['data_preprocessing']['version']) 
model_output_path = "/".join([os.getcwd(),config_catalog['output_file_path']['model_output_path'],version])
newpath = model_output_path 
processed_data_path =  "/".join([newpath,'processed_data'])
final_results_path =  "/".join([newpath,'final_results'])
plot_summary_path =  "/".join([newpath,'plot_summary'])
# Output Paths
if not os.path.exists(newpath):
    os.makedirs(newpath)
    os.makedirs(processed_data_path)
    os.makedirs(final_results_path)
    os.makedirs(plot_summary_path)
   

# Inputs
input_data =  "/".join([os.getcwd(),config_catalog['data_preprocessing']['actual_file_path']])


Load Parameters

In [13]:

increa_afterblowdown_kettle = config_parameters['optimize_engineSettings']['optimizer']['incremental_afterblowdown_kettle']
increa_blowdown_qty = config_parameters['optimize_engineSettings']['optimizer']['incremental_blowdown_qty']
function_ =  config_parameters['optimize_engineSettings']['optimizer']['function_']

if function_ == 'linear' : functions = config_parameters['optimize_engineSettings']['optimizer']['functions']['linear']
else: functions = config_parameters['optimize_engineSettings']['optimizer']['functions']['non_linear']

multiply =  config_parameters['optimize_engineSettings']['optimizer']['multiply']
drop_per_iteration =  config_parameters['optimize_engineSettings']['optimizer']['drop_per_iteration']
min_thresh =  config_parameters['optimize_engineSettings']['optimizer']['min_thresh']
iterations =  config_parameters['optimize_engineSettings']['optimizer']['iterations']
sequence = config_parameters['optimize_engineSettings']['optimizer']['sequence']

opt_parameters  = config_parameters['optimize_engineSettings']['optimizer']

print(f"Optimization parameters == function_:{function_, functions} || min_thresh:{min_thresh} || multiply:{multiply} || trials:{iterations} || drop_per_iteration:{drop_per_iteration}")

Optimization parameters == function_:('linear', {'intercept': -0.4831, 'b1': 0.7251}) || min_thresh:30 || multiply:100 || trials:10 || drop_per_iteration:10


In [6]:
# To create sequence
data_30 = generate_opt_data(18,processed_data_path)

In [7]:
opt_df = data_30.copy() 
opt_df = pd.concat([opt_df,opt_df.loc[3:],opt_df.loc[3:],opt_df.loc[3:],opt_df.loc[3:],opt_df.loc[3:],opt_df.loc[3:]]).reset_index(drop=True)

In [8]:
cols_to_opt = opt_df.columns.to_list()

In [89]:
best_params_dic, study_opt , opt_datafame = wrapper(opt_df, cols_to_opt, iterations, increa_afterblowdown_kettle, increa_blowdown_qty, opt_parameters, function_ = function_, min_thresh = min_thresh, multiply=100, drop_per_iteration = drop_per_iteration )
opt_datafame.columns = ['BD qty', 'KL after BD', 'HB', 'NO BD', 'Sequence ID', 'Revenue']
# opt_datafame.to_csv(os.path.join(final_results_path,'optimized_top_10_latest.csv'),index=False)

In [10]:
seq = opt_datafame[opt_datafame['Revenue']== opt_datafame['Revenue'].max()]['Sequence ID'].unique()[0]
test = opt_datafame[opt_datafame['Sequence ID']== seq]
test = pd.concat([test[test['Revenue']==x] for x in sorted(list(test['Revenue'].unique()))])

In [14]:
animation, optimization_history, parallel_coordinate, feature_importance, edf = plot_animation(test,study_opt,seq)

In [118]:
with open(os.path.join(plot_summary_path,'optimization_summary.html'), 'a') as f:
    f.write(animation.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(optimization_history.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(parallel_coordinate.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(feature_importance.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(edf.to_html(full_html=False, include_plotlyjs='cdn'))